Standortkarten Print Skript

In [1]:
import sys
import os
import psycopg2
import time
import configparser
import requests
import json
import subprocess
from requests.auth import HTTPBasicAuth
import urllib2
from ipywidgets import FloatProgress
from IPython.display import display

In [2]:
def process_exists(process_name):
    call = 'TASKLIST', '/FI', 'imagename eq %s' % process_name
    output = subprocess.check_output(call)
    last_line = output.strip().split('\r\n')[-1]
    return last_line.lower().startswith(process_name.lower())

DatenbankServerFunktionen
@todo: Auslagern in eine Klasse

In [3]:
def openDataBaseServer(config):
    if not process_exists("postgres.exe"):
        print "Datenbank ist nicht offen!"
        processcall=config.get("postgres","bin_dir")+"/bin/postgres.exe "
        processcall+="-D "+config.get("postgres","db_dir")+" "
        processcall+="-p "+config.get("postgres","port")
        
        subprocess.Popen(processcall,shell=True)
        time.sleep(10)
        #@todo check if really open!
        print("Datenbank offen!")

In [4]:
def killDataBaseServer():
    p1=subprocess.Popen("TASKKILL /IM postgres.exe > NUL 2>&1",shell=True)
    p1.wait()
    print "Datenbankserver beendet"

In [5]:
def initDataBaseServer(config):
    if os.path.isdir(config.get("postgres","db_dir"))==False:
        print "Datenbank ist nicht eingerichtet, richte ein!"
        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/initdb --locale=German_Germany.1252 --encoding=UTF8 "
        processcall+=config.get("postgres","db_dir")
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()
        datenbank=openDataBaseServer(config)
        processcall=config.get("postgres","bin_dir")+"/bin/createuser.exe -s -d -w "+config.get("postgres","user")
        p1=subprocess.Popen(processcall,shell=True)
        p1.wait()
        print "Nutzer eingerichtet"

In [6]:
def download_file(url,dest):
    urllib2.install_opener(
            urllib2.build_opener(
                urllib2.ProxyHandler({'https': config.get("general","proxy_https")})
            )
        )
        
    file_name = "tempdata/"+osm_url.split('/')[-1]
    u = urllib2.urlopen(osm_url)
    f = open(file_name, 'wb')
    meta = u.info()
    file_size = int(meta.getheaders("Content-Length")[0])
    print "Downloading: %s Bytes: %s" % (file_name, file_size)

    file_size_dl = 0
    block_sz = 8192

    progressbar = FloatProgress(min=0, max=100) # instantiate the bar
    display(progressbar) # display the bar

    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break
        file_size_dl += len(buffer)
        f.write(buffer)
        progressbar.value=file_size_dl * 100. / file_size
    f.close()

In [9]:
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = stdout
#disable proxy for localhost
os.environ['NO_PROXY'] = '127.0.0.1'
config=configparser.ConfigParser()

In [10]:
try:
    config.read("printconfig.ini")
except:
    print "exception while reading config file"

In [21]:
initDataBaseServer(config)
openDataBaseServer(config)

#beispielhaft für Baden_Württemberg
#DB ist OSM_BW_1
print config.get("1","osm_url")
osm_url=config.get("1","osm_url")
databasename="osm_bw_1"
try:
    conn = psycopg2.connect("dbname='"+databasename+"' user='"
                            +config.get("postgres","user")
                            +"' host='"+config.get("postgres","host")
                            +"' password='"+config.get("postgres","password")+"'")
    print "Datenbank existiert"
except:
    print "Datenbank existiert nicht!"
    #download osm data if not existent
    if not os.path.exists("tempdata"):
        os.makedirs("tempdata")
    if not os.path.isfile("tempdata/"+os.path.basename(osm_url)):
        print "downloading OSM file"
        download_file(osm_url,"tempdata/"+os.path.basename(osm_url))
        print "Download abgeschlossen"
    #create database
    processcall=config.get("postgres","bin_dir")
    processcall+="/bin/psql -w -U "+config.get("postgres","user")
    processcall+=" -p "+config.get("postgres","port")
    processcall+=" -d postgres -c \"CREATE DATABASE "+databasename+"\""
    
    print processcall
    p1 = subprocess.Popen(processcall,shell=True)
    p1.wait()

    processcall=config.get("postgres","bin_dir")
    processcall+="/bin/psql -w -U "+config.get("postgres","user")
    processcall+=" -d \""+databasename+"\""
    processcall+=" -c \"CREATE EXTENSION postgis;\""
    p1 = subprocess.Popen(processcall,shell=True)
    p1.wait()

    processcall=config.get("postgres","bin_dir")
    processcall+="/bin/psql -w -U "+config.get("postgres","user")
    processcall+=" -d \""+databasename+"\""
    processcall+=" -c \"CREATE EXTENSION postgis_sfcgal;\""
    p1 = subprocess.Popen(processcall,shell=True)
    p1.wait()

    processcall=config.get("postgres","bin_dir")
    processcall+="/bin/psql -w -U "+config.get("postgres","user")
    processcall+=" -d \""+databasename+"\""
    processcall+=" -c \"CREATE EXTENSION postgis_topology;\""
    p1 = subprocess.Popen(processcall,shell=True)
    p1.wait()

    processcall=config.get("postgres","bin_dir")
    processcall+="/bin/psql -w -U "+config.get("postgres","user")
    processcall+=" -d \""+databasename+"\""
    processcall+=" -c \"CREATE EXTENSION hstore;\""
    p1 = subprocess.Popen(processcall,shell=True)
    p1.wait()
    
    #import data
    processcall=config.get("osm2pgsql","bin_path")
    processcall+=" -d "+databasename
    processcall+=" -S "+config.get("osm2pgsql","schema")
    processcall+=" -k --hstore-match-only -r pbf -s"
    processcall+=" -H "+config.get("postgres","host")
    processcall+=" -p "+config.get("postgres","port")
    processcall+=" -U "+config.get("postgres","user")
    processcall+=" -C "+config.get("osm2pgsql","cachesize")
    processcall+" -c  \""+os.path.abspath("tempdata/"+os.path.basename(osm_url))+"\""
    
    print processcall

https://download.geofabrik.de/europe/germany/baden-wuerttemberg-latest.osm.pbf
Datenbank existiert


In [ ]:
r=requests.post(config.get("webservice","url"),
                auth=HTTPBasicAuth(config.get("webservice","username"),config.get("webservice","password")),
                data={"command":"fetchAllPrintingQueue"},
               )
if r.status_code == 200:
    queue=json.loads(r.text)
    for job in queue:
        print job
        #start db and wait some seconds
        
        #import mapnik
        sys.path.append(config.get("mapnik","pythonPath")) 
        import mapnik
        
else:
    print "konnte Printqueue nicht holen",r.status_code,r.text
